                   Bank Marketing Data Set https://archive.ics.uci.edu/ml/datasets/bank+marketing
1,The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. There are more than 11000 examples and 17 attributes,like age,gender,if has personal loan etc., ordered by date (from May 2008 to November 2010).                  

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

2,load the dataset and binarize those features

In [56]:
df = pd.read_csv('bank.csv')
print(df.shape)
df.head()

(11162, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


binarize countious features

In [57]:
def binarize_num(name):
    for i in df[name]:
        if i < df[name].mean():
            df[name] = 0
        else:
            df[name] = 1

In [58]:
name_lst = ['age','balance','day','duration','campaign','pdays','previous']
for name in name_lst:
    binarize_num(name)

binarize category features

In [59]:
df2 = pd.get_dummies(df, prefix=['job','marital','education','contact','month','poutcome'], columns=
              ['job','marital','education','contact','month','poutcome'])

replace deposit label 'yes' with 1, 'no' with 0

In [60]:
df3 = df2.replace(['yes','no'],[1,0])
print(df3.shape)
df3.head()

(11162, 49)


,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,1,0,1,1,0,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,1
1,1,0,1,0,0,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,1
2,1,0,1,1,0,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,1
3,1,0,1,1,0,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,1
4,1,0,1,0,0,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,1


3,create a train-test split, using 2/3 for train and 1/3 for test.

In [61]:
y = df3['deposit']
X = df3.drop(['deposit'], axis=1)
print(X.shape,y.shape)

(11162, 48) (11162,)


In [62]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.333,random_state=123)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(7445, 48) (7445,) (3717, 48) (3717,)


4,Train a BernouilliNB classifier on the train split; use the default parameters.

In [63]:
bnb = BernoulliNB()
bnb = bnb.fit(X_train,y_train)

5, Find 5 type objects, and print a) the total positive log-evidence,  b) the total negative log-evidence, c) probability distribution, d) top 3 features values that contribute most to the positive evidence, e) top 3 feature values that contribute the most to the negative evidence. 

If we explore the results of probability:

In [64]:
prob_res = bnb.predict_proba(X_test)
print('sort the positive objs with probability',np.argsort(prob_res[:,1]))

sort the positive objs with probability [3262  152 2147 ..., 3465 1742  951]


In [65]:
print('the probability of the most positive object is:', prob_res[np.argsort(prob_res[:,1])][-1,:])

the probability of the most positive object is: [  1.76157669e-04   9.99823842e-01]


In [66]:
print('sort the negative objs with probability',np.argsort(prob_res[:,0]))

sort the negative objs with probability [ 951 1742 3465 ...,  152 2147 3262]


In [67]:
print('the probability of the most negative object is:', prob_res[np.argsort(prob_res[:,0])][-1,:])

the probability of the most negative object is: [ 0.9960786  0.0039214]


If we explore the result of log probability:

In [68]:
log_prob_res = bnb.predict_log_proba(X_test)
positive_diff = log_prob_res[:,1]-log_prob_res[:,0]

In [69]:
print('sort the positive objs with log probability',np.argsort(positive_diff))

sort the positive objs with log probability [3262  152 2147 ..., 3465 1742  951]


From above result, we can see the most positive object with respect to the probabilities is just the object that has the largest positive evidence: the 3262th object, and the most negative object with respect to the probabilities is	the object that has the largest (in magnitude) negative evidence: 951th obj.

In [70]:
type13 = X_test.iloc[[3262]]
type24 = X_test.iloc[[951]]

To calculate total evidence, we need to consider the features' probability

In [71]:
pos_lst = bnb.feature_log_prob_[1,:]
neg_lst = bnb.feature_log_prob_[0,:]

In [72]:
print('total positive log-evidence for (1) and (3):',positive_diff[np.argsort(positive_diff)][-1])
print('total negative log-evidence for (2) and (4):',positive_diff[np.argsort(positive_diff)][0])

total positive log-evidence for (1) and (3): 8.64395494555
total negative log-evidence for (2) and (4): -5.53737655943


In [73]:
print('the probability distribution for (1) and (3):', prob_res[np.argsort(prob_res[:,1])][-1,:])
print('the probability distribution for (2) and (4):', prob_res[np.argsort(prob_res[:,0])][-1,:])

the probability distribution for (1) and (3): [  1.76157669e-04   9.99823842e-01]
the probability distribution for (2) and (4): [ 0.9960786  0.0039214]


The most uncertain objects are those with same probabilities. It should be the 1752th instance.

In [74]:
np.argsort(np.abs(positive_diff), axis=-1)

array([1752,  157, 1322, ..., 3465, 1742,  951], dtype=int64)

In [75]:
type5 = X_test.iloc[[1752]]

In [76]:
top3_pos = np.argsort(pos_lst)[:3]
col_lst = list(X_test.columns)
print('top 3 features contribute most to positive evidence',[col_lst[i] for i in top3_pos])

top 3 features contribute most to positive evidence ['job_unknown', 'default', 'month_dec']


In [77]:
top3_neg = np.argsort(neg_lst)[:3]
print('top 3 features contribute most to negative evidence',[col_lst[i] for i in top3_neg])

top 3 features contribute most to negative evidence ['month_dec', 'month_mar', 'job_unknown']


In [78]:
print('the probability distribution for (5):', prob_res[1752])

the probability distribution for (5): [ 0.49935626  0.50064374]
